# LSTM

In [49]:
import Pkg; #Pkg.add("IterTools"); Pkg.add("Knet"); Pkg.add("Plots"); Pkg.add("CUDA"); Pkg.add("PlotlyJS")
using Knet
using Printf
using Statistics: mean
using Base.Iterators: flatten
using IterTools: ncycle, takenth
import .Iterators: cycle, Cycle, take
using Plots; plotlyjs()
using DelimitedFiles, Statistics, Random
import CUDA
array_type=(CUDA.functional() ? KnetArray{Float32} : Array{Float32})
dataset_directory_path = "../DATA"
dataset_directory = filter(x -> isdir(joinpath(dataset_directory_path, x)), readdir(dataset_directory_path))

7-element Array{String,1}:
 "doganay_experiment"
 "idil_experiment"
 "istenc_experiment"
 "omer_experiment"
 "ozan_experiment"
 "yahya_experiment"
 "zaid_experiment"

In [65]:
send_message(message) = run(`curl https://notify.run/zrqfxE0M1ypL5Dv4 -d $message`);

In [66]:
train_data = Knet.load("raw_data.jld2","train");
test_data = Knet.load("raw_data.jld2","test");

In [67]:
function split_and_convert(string_array,splitter)
    Rlength = length(string_array)
    Clength = length(split(string_array[1],splitter))
    Mdata = zeros(Float64,Rlength,Clength)
    for i in 1:Rlength
        row =  map(x->tryparse(Float64,x),split(string_array[i],splitter))
        Mdata[i,:] = row
    end
    return Mdata
end

split_and_convert (generic function with 1 method)

In [68]:
time_history = 50;

In [69]:
function data_selection(participant_directory_path)
    data_matrix = Array{Float32}(undef,0,9);
    participant_directory = readdir(participant_directory_path)
    for raw_txt_files in participant_directory
            txt_file_directory_path = joinpath(participant_directory_path, raw_txt_files)
            txt_file_directory = readdir(txt_file_directory_path)
            
            raw_data_file = open(joinpath(txt_file_directory_path,txt_file_directory[4]))
            raw_data = split_and_convert(readlines(raw_data_file),",")
            close(raw_data_file)
            
            #To avoiding the colliding between subtask 3 and subtask 1
            _, remain = divrem(size(raw_data,1),time_history)
            
            #  Velocity F_fint  F_h  subtask 
            #  1        2       3      4     
            data_matrix = [data_matrix; raw_data[1:end-remain,:]]
        end
        
    return data_matrix
end

data_selection (generic function with 1 method)

In [70]:
test_data = Array{Float32}(undef,0,9);
train_data = Array{Float32}(undef,0,9);

for participant_index in 1:length(dataset_directory)
    #Participants: 1.doganay 2.idil 3.istenc 4.omer 5.ozan 6.yahya 7.zaid
    participant_directory_path = joinpath(dataset_directory_path,dataset_directory[participant_index])
    participant_directory = readdir(participant_directory_path)
    
    if participant_index != 5
        data = data_selection(participant_directory_path)
        train_data = [train_data;data]
    else
        data = data_selection(participant_directory_path)
        test_data = [test_data;data]
    end
end

Sorting by Length (Descending)

In [71]:
#max_, _ = findmax(list_series)
#min_ , _ = findmin(list_series)
#mean_ = mean(list_series)
#print("Max: ",max_," Min: ", min_ ," Mean: ", mean_)
# Adding Related Time Series Length to the DATA
#assign_durations(data_array,duration_list) = for i in 1:size(data_array)[1]; data_array[i] = [data_array[i],duration_list[i]];end
##For the train_data
#assign_durations(train_data,list_series[1:length(train_data)])
##For the test_data
#assign_durations(test_data,list_series[length(train_data)+1:end])
# Sorting Data
#sort!(train_data, by = x -> x[:][2],rev=true); 
#sort!(test_data, by = x -> x[:][2],rev=true); 

In [72]:
mutable struct pHRIHelper
   measurements
   subtasks
   batchsize
   time_history
   ninstances
   shuffled
end

function data_splitter(raw_data)
    input = [raw_data[:,2] raw_data[:,3] raw_data[:,5]];
    output = raw_data[:,end];
    return input,output
end


function pHRIHelper(raw_data; batchsize = 128, time_history = 50, shuffled = true)
   measurements, subtasks = data_splitter(raw_data)
   ninstances, _ = divrem(length(subtasks),time_history+batchsize-2)
   ninstances = convert(Int,ninstances)
   return pHRIHelper(measurements,subtasks,batchsize,time_history,ninstances,shuffled)
end

pHRIHelper

In [73]:
#State data stands for ninstances, But every iteration it gets 50 timesteps for each batchsize element.
#Each element 

function Base.iterate(d::pHRIHelper, state=ifelse(d.shuffled, randperm(d.ninstances), 1:d.ninstances))
    output = KnetArray(zeros(Int32,d.batchsize,d.time_history));        #time_steps,1
    input = KnetArray(zeros(Float32,3,d.batchsize,d.time_history));        # X B T -> 3x16x50
    needed_volume = d.batchsize+d.time_history-2;
    related_measurements = d.measurements[state[1]:state[1]+needed_volume,:]
    related_subtasks = d.subtasks[state[1]:state[1]+needed_volume];
    buffer = zeros(time_history,1);
    x_matrix = zeros(3,time_history);
    y_matrix = zeros(1,time_history);
    for time_step in 1:d.batchsize
        buffer = time_step:time_step+time_history-1;
        x_matrix = related_measurements[buffer,:];
        y_matrix = related_subtasks[buffer];
        input[:,time_step,:] = x_matrix';
        output[time_step,:] = y_matrix;
    end
    new_state = state[2:end]
    length(new_state) < 1 && return nothing
    return ((input, output),new_state)    
end

In [74]:
((input, output),new_state) = iterate(dtrn)

LoadError: Out of GPU memory trying to allocate 75.000 KiB
Effective GPU memory usage: 99.97% (1.999 GiB/2.000 GiB)
CUDA allocator usage: 1.526 GiB
binned usage: 116.578 KiB (116.578 KiB allocated, 0 bytes cached)
Discrepancy of 1.526 GiB between memory pool and allocator!


In [75]:
mutable struct Linear
    w # weight
    b # bias

    function Linear(inputsize, outputsize; atype=array_type, scale::Float64=0.01)
        new(Param(convert(atype,randn(outputsize,inputsize)*scale)),Param(convert(atype,zeros(outputsize,1))))
    end
end

function (l::Linear)(x)
    return l.w*x.+l.b;
end

mutable struct Hidden
    w # weight
    b # bias
    fun # non-linear activation function like relu or tanh

    function Hidden(inputsize, outputsize, fun=relu, atype=array_type, scale=0.1)
        w = Param(convert(atype,randn(outputsize,inputsize)*scale));
        b = Param(convert(atype,zeros(outputsize,1)));
         new(w,b,fun)
    end
end

function (l::Hidden)(x)
    return l.fun.(l.w*x.+l.b);
end

In [76]:
mutable struct pHRITrainer
    hidden_input::Hidden
    rnn::RNN
    hidden::Linear
end

function pHRITrainer(input_size, hidden_size, output_size, atype=array_type)
    hidden_input = Hidden(input_size,hidden_size)
    rnn = RNN(hidden_size,hidden_size; numLayers=2,rnnType=:lstm);
    hidden = Linear(hidden_size,output_size)
    return pHRITrainer(hidden_input,rnn, hidden)
end 

function (m::pHRITrainer)(x)
    dim = size(x)
    a = m.hidden_input(reshape(x,size(x,1),:))
    b = m.rnn(reshape(a,size(a,1),dim[2],dim[3]));
    c = m.hidden(reshape(b,size(b,1),:));
    output = reshape(c,size(c,1),dim[2],dim[3]);
    return output;
end

function (m::pHRITrainer)(data::pHRIHelper)
    total_loss = 0
    count = 0
    for (x,y) in data
        (loss,number) = nll(model(x),y;average=false)
        total_loss += loss
        count += number
    end
    return total_loss/count
end

In [77]:
function loss(model::pHRITrainer, data; average=true)
    total_loss = 0
    count = 0
    for (x,y) in data
        (loss,number) = nll(model(x),y;average=false)
        total_loss += loss
        count += number
    end
    average && return total_loss/count
    return total_loss, count
end

function accurasix(model::pHRITrainer, data)
    ncorrect = 0
    ntokens = 0
    for (x, ygold) in data
        scores = model(x)
        ntokens += length(ygold)
        ypred = map( y -> y[1], argmax(scores, dims=1))
        ncorrect += sum(ygold .== ypred')
    end
    return ncorrect / ntokens
end

accurasix (generic function with 1 method)

In [78]:
function train!(model, trn, dev, tst...)
    bestmodel, bestloss = deepcopy(model), loss(model, dev)
    progress!(adam(model, trn), steps=1000) do y losses = [ loss(model, d) for d in (dev,tst...) ]
        if losses[1] < bestloss
            bestmodel, bestloss = deepcopy(model), losses[1]
        end
        return (losses...,)
    end
    return bestmodel
end

train! (generic function with 1 method)

In [79]:
Random.seed!(1)

dtrn = pHRIHelper(train_data;shuffled=false);
dtst = pHRIHelper(test_data;shuffled=false);
model = pHRITrainer(3, 128, 3)
epochs=1;

ctrn = []
for b in dtrn
   push!(ctrn,b) 
end

trnx10 = collect(flatten(ctrn for i in 1:epochs))

trn20 = []
counter = 0
for d in dtrn
    if counter > 20 
        break
    end
    push!(trn20,d)
    counter +=1
end

dev = []
for b in dtst
   push!(dev,b) 
end

model = train!(model, trnx10, dev, trn20)

#display(model(dtrn))
#display(nll(model;data=dtrn))

LoadError: Out of GPU memory trying to allocate 1.008 MiB
Effective GPU memory usage: 99.97% (1.999 GiB/2.000 GiB)
CUDA allocator usage: 1.526 GiB
binned usage: 116.594 KiB (116.594 KiB allocated, 0 bytes cached)
Discrepancy of 1.526 GiB between memory pool and allocator!


In [48]:
function train!(model, trn, dev, tst...)
    bestmodel, bestloss = deepcopy(model), loss(model, dev)
    progress!(adam(model, trn), steps=1000) do y
        losses = [ loss(model, d) for d in (dev,tst...) ]
        if losses[1] < bestloss
            bestmodel, bestloss = deepcopy(model), losses[1]
        end
        return (losses...,)
    end
    return bestmodel
end

function mytrain!(model::pHRITrainer, train_data::pHRIHelper, test_data::pHRIHelper,period::Int=10, iters::Int=500)
    train_loss, test_loss = Any[],Any[]
    for i in 0:period:iters
        train_loss = [train_loss; model(train_data)]
        test_loss = [test_loss; model(test_data)]
        sgd!(model,take(cycle(train_data),period))
    end
    return 0:period:iters, train_loss, test_loss
end

iters, trnloss, tstloss = train!(model, dtrn, dtst)
@info "mytrain! tests passed"
plot(iters, trnloss, label="train", xlabel="iterations", ylabel="loss")
plot!(iters, tstloss, label="test")

LoadError: Out of GPU memory trying to allocate 34.375 MiB
Effective GPU memory usage: 98.36% (1.967 GiB/2.000 GiB)
CUDA allocator usage: 1.466 GiB
binned usage: 101.938 KiB (101.938 KiB allocated, 0 bytes cached)
Discrepancy of 1.466 GiB between memory pool and allocator!


In [ ]:
for x in takenth(progress(adam(model,lr=0.15,ncycle(dtrn,100))),dtrn.ninstances); display(x) end


In [ ]:
send_message("Done!")

In [ ]:
function mytrain!(model::pHRITrainer, dtrn::pHRIHelper, dtst::pHRIHelper; lr_input=0.15, epoch::Int=100, optimizer)
    train_loss = Array{Float64,1}()
    test_loss = Array{Float64,1}()
    train_error = Array{Float64,1}()
    test_error = Array{Float64,1}()
    for x in takenth(progress(optimizer(model,lr=lr_input,ncycle(dtrn,epoch))),dtrn.ninstances)
        push!(train_loss,model(dtrn))
        push!(test_loss,model(dtst))
        push!(train_error, 1-accuracy(model,data = dtrn))
        push!(test_error, 1-accuracy(model,data = dtst))
    end
    return 1:epoch, train_loss, test_loss, train_error, test_error
end

function train_n_show(model,dtrn,dtst,epoch_,learning_rate,optimizer_,Title)
    epoch = Array{Int64,1}()    
    trnloss = Array{Float64,1}()
    tstloss = Array{Float64,1}()
    trnerror = Array{Float64,1}()
    tsterror = Array{Float64,1}()
    epoch, trnloss, tstloss, trnerror, tsterror = mytrain!(model,dtrn,dtst,epoch = epoch_,lr_input = learning_rate ,optimizer=optimizer_);
    loss_graph = plot(epoch, trnloss, label="Training Loss" , xlabel = "epochs", ylabel = "loss", title=Title)
    plot!(epoch, tstloss, label="Test Loss")
    error_graph = plot(epoch, trnerror, label="Training Error" , xlabel = "epochs", ylabel = "error", title=Title)
    plot!(epoch, tsterror, label="Test Error")
    display(loss_graph)
    display(error_graph)
    print("Training Loss: ",round(minimum(trnloss),digits=4)," Test Loss: ", round(minimum(tstloss),digits=4))
    print("\nTraining Error: ",round(minimum(trnerror),digits=4)," Test Error: ", round(minimum(tsterror),digits=4))
end


iters, trnloss, tstloss = train_n_show(model, dtrn, dtst,80,0.0001,adam,"Let's See")

In [ ]:
cycle(dtrn)